In [1]:
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u

from galpy.orbit import Orbit
from astroML.density_estimation import XDGMM

## Load Data

In [2]:
data_ic = Table.read("data/outputs/ic348_apogee.csv")
data_ngc = Table.read("data/outputs/ngc1333_apogee.csv")

In [3]:
# sun orbit data -- used later to convert between uvw and UVW
osun = Orbit([0,0,0,-11.1, -12.24, -7.25],radec=True,uvw=True)

## Functions

##### calculate LBD values for each star

In [4]:
def calculate_LBDUVW(row):
    samp = np.array([row['ra'], row['dec'], row['parallax'], row['pmra'], row['pmdec'], row['RV']])
    
    coord = SkyCoord(ra = samp[0] * u.deg, dec = samp[1] * u.deg, distance=1000/samp[2] *u.pc,
                             pm_ra_cosdec=samp[3] *u.mas/u.yr, pm_dec=samp[4] *u.mas/u.yr,
                             radial_velocity=samp[5] *u.km/u.s,
                             frame='icrs')
    
    o = Orbit(coord)
    
    return coord.galactic.l.value, coord.galactic.b.value, coord.galactic.distance.value, o.U(), o.V(), o.W()

##### calculate LBD and UVW errors for each star

In [5]:
def calculate_err(row):
    mean = np.array([row['ra'], row['dec'], row['parallax'], row['pmra'], row['pmdec'], row['RV']])
    std = np.array([row['ra_error'], row['dec_error'], row['parallax_error'], row['pmra_error'], row['pmdec_error'], row['e_RV']])
    
    X = np.vstack(mean)
    Xerr = np.zeros((6,6))
    np.fill_diagonal(Xerr, std)
    
    single_star_samps = np.random.multivariate_normal(mean, Xerr**2, size=100, check_valid='warn', tol=1e-8)
    
    coords = []
    for samp in single_star_samps:
        if samp[2] < 0:
            pass
        else:
            coords.append(SkyCoord(ra = samp[0] * u.deg, dec = samp[1] * u.deg, distance=1000/samp[2] *u.pc,
                             pm_ra_cosdec=samp[3] *u.mas/u.yr, pm_dec=samp[4] *u.mas/u.yr,
                             radial_velocity=samp[5] *u.km/u.s,
                             frame='icrs'))
    
    l_list = []
    d_list = []
    b_list = []
    U_list = []
    V_list = []
    W_list = []

    for coord in coords:
        l_list.append(coord.galactic.l.value)
        b_list.append(coord.galactic.b.value)
        d_list.append(coord.galactic.distance.value)
        
        o = Orbit(coord)
        U_list.append(o.U())
        V_list.append(o.V())
        W_list.append(o.W())
        
    l_err = np.std(l_list)
    b_err = np.std(b_list)
    d_err = np.std(d_list)
    
    U_err = np.std(U_list)
    V_err = np.std(V_list)
    W_err = np.std(W_list)

    
    return l_err, b_err, d_err, U_err, V_err, W_err

## Calculate Coords and Errors for Clusters

In [6]:
l_err_ic = []
b_err_ic = []
d_err_ic = []

U_err_ic = []
V_err_ic = []
W_err_ic = []

L_ic = []
B_ic = []
D_ic = []

U_ic = []
V_ic = []
W_ic = []

count = 1

for row in data_ic:
    print(count)
    l_err, b_err, d_err, U_err, V_err, W_err = calculate_err(row)
    L,B,D, U, V, W= calculate_LBDUVW(row)
    
    l_err_ic.append(l_err)
    b_err_ic.append(b_err)
    d_err_ic.append(d_err)
    
    U_err_ic.append(U_err)
    V_err_ic.append(V_err)
    W_err_ic.append(W_err)
    
    L_ic.append(L)
    B_ic.append(B)
    D_ic.append(D)
    
    U_ic.append(U)
    V_ic.append(V)
    W_ic.append(W)
    
    count += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166


In [7]:
l_err_ngc = []
b_err_ngc = []
d_err_ngc = []

U_err_ngc = []
V_err_ngc = []
W_err_ngc = []

L_ngc = []
B_ngc = []
D_ngc = []

U_ngc = []
V_ngc = []
W_ngc = []

count = 1

for row in data_ngc:
    print(count)
    l_err, b_err, d_err, U_err, V_err, W_err = calculate_err(row)
    L,B,D,U,V,W = calculate_LBDUVW(row)
    
    l_err_ngc.append(l_err)
    b_err_ngc.append(b_err)
    d_err_ngc.append(d_err)
    
    U_err_ngc.append(U_err)
    V_err_ngc.append(V_err)
    W_err_ngc.append(W_err)
    
    L_ngc.append(L)
    B_ngc.append(B)
    D_ngc.append(D)
    
    U_ngc.append(U)
    V_ngc.append(V)
    W_ngc.append(W)
    
    count += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


## Running Deconvolution

##### IC 348

In [8]:
X_ic = np.vstack([L_ic, B_ic, D_ic, U_ic, V_ic, W_ic]).T
Xerr_ic = np.zeros(X_ic.shape + X_ic.shape[-1:])

for i in range(0,len(l_err_ic)):
    np.fill_diagonal(Xerr_ic[i], [l_err_ic[i]**2, b_err_ic[i]**2, d_err_ic[i]**2,
                              U_err_ic[i]**2, V_err_ic[i]**2, W_err_ic[i]**2])

clf_ic = XDGMM(n_components = 2)
clf_ic.fit(X_ic, Xerr_ic)

mu_ic = clf_ic.mu
cov_ic = clf_ic.V
alpha_ic = clf_ic.alpha

In [9]:
# select main cluster
amp_mask_ic = alpha_ic > .5

mean_ic = mu_ic[amp_mask_ic][0]
sigma_ic = np.sqrt(np.diagonal(cov_ic[amp_mask_ic][0]))

##### NGC 1333

In [10]:
X_ngc = np.vstack([L_ngc, B_ngc, D_ngc, U_ngc, V_ngc, W_ngc]).T
Xerr_ngc = np.zeros(X_ngc.shape + X_ngc.shape[-1:])

for i in range(0,len(l_err_ngc)):
    np.fill_diagonal(Xerr_ngc[i], [l_err_ngc[i]**2, b_err_ngc[i]**2, d_err_ngc[i]**2,
                              U_err_ngc[i]**2, V_err_ngc[i]**2, W_err_ngc[i]**2])

clf_ngc = XDGMM(n_components = 2)
clf_ngc.fit(X_ngc, Xerr_ngc)

mu_ngc = clf_ngc.mu
cov_ngc = clf_ngc.V
alpha_ngc = clf_ngc.alpha

In [11]:
# select main cluster
amp_mask_ngc = alpha_ngc > .5

mean_ngc = mu_ngc[amp_mask_ngc][0]
sigma_ngc = np.sqrt(np.diagonal(cov_ngc[amp_mask_ngc][0]))

## Deconvolution Results

In [13]:
print("IC 348 summary")
print(f"means   L:{mean_ic[0]} --  B:{mean_ic[1]} -- D:{mean_ic[2]}")
print(f"        U:{mean_ic[3]} --  V:{mean_ic[4]} --  W:{mean_ic[5]}")
print(f"        u:{mean_ic[3] - osun.U()} -- v:{mean_ic[4] - osun.V()} -- w:{mean_ic[5] - osun.W()}")
print(f"sigma   L:{sigma_ic[0]} --  B:{sigma_ic[1]} -- D:{sigma_ic[2]}")
print(f"        U:{sigma_ic[3]} --  V:{sigma_ic[4]} --  W:{sigma_ic[5]}")

print("")

print("NGC 1333 summary")
print(f"means   L:{mean_ngc[0]} --  B:{mean_ngc[1]} -- D:{mean_ngc[2]}")
print(f"        U:{mean_ngc[3]} --  V:{mean_ngc[4]} --  W:{mean_ngc[5]}")
print(f"        u:{mean_ngc[3] - osun.U()} -- v:{mean_ngc[4] - osun.V()} -- w:{mean_ngc[5] - osun.W()}")
print(f"sigma   L:{sigma_ngc[0]} --  B:{sigma_ngc[1]} -- D:{sigma_ngc[2]}")
print(f"        U:{sigma_ngc[3]} --  V:{sigma_ngc[4]} --  W:{sigma_ngc[5]}")


IC 348 summary
means   L:160.4892413425805 --  B:-17.80876937457776 -- D:315.2470816561538
        U:-16.701330889480783 --  V:-6.145498186382202 --  W:-7.582557065016388
        u:-5.601330889480785 -- v:6.094501813617823 -- w:-0.33255706501639004
sigma   L:0.08494058638784946 --  B:0.05237350424882814 -- D:7.103391755451349
        U:1.1102263132510637 --  V:1.165861951866328 --  W:0.9606088985480086

NGC 1333 summary
means   L:158.3159702616971 --  B:-20.472365874315546 -- D:296.720765776094
        U:-16.835271565656218 --  V:-10.984136986743056 --  W:-9.736973336851882
        u:-5.73527156565622 -- v:1.2558630132569686 -- w:-2.486973336851884
sigma   L:0.05039372944450675 --  B:0.07191893183272671 -- D:8.862305589804084
        U:1.0129897927427556 --  V:1.603970704666745 --  W:0.7959778899041949


## Exporting Results

In [18]:
t = Table()
t["cluster"] = ["IC 348", "NGC 1333"]
t["L"] = [mean_ic[0], mean_ngc[0]]
t["B"] = [mean_ic[1], mean_ngc[1]]
t["D"] = [mean_ic[2], mean_ngc[2]]
t["L_err"] = [sigma_ic[0], sigma_ngc[0]]
t["B_err"] = [sigma_ic[1], sigma_ngc[1]]
t["D_err"] = [sigma_ic[2], sigma_ngc[2]]
t["U"] = [mean_ic[3], mean_ngc[3]]
t["V"] = [mean_ic[4], mean_ngc[4]]
t["W"] = [mean_ic[5], mean_ngc[5]]
t["U_err"] = [sigma_ic[3], sigma_ngc[3]]
t["V_err"] = [sigma_ic[4], sigma_ngc[4]]
t["W_err"] = [sigma_ic[5], sigma_ngc[5]]
t["UVW_dispersion"] = [np.sqrt(sigma_ic[3]**2 + sigma_ic[4]**2 + sigma_ic[5]**2), 
                       np.sqrt(sigma_ngc[3]**2 + sigma_ngc[4]**2 + sigma_ngc[5]**2)]
t["u"] = [mean_ic[3] - osun.U(), mean_ngc[3] - osun.U()]
t["v"] = [mean_ic[4] - osun.V(), mean_ngc[4] - osun.V()]
t["w"] = [mean_ic[5] - osun.W(), mean_ngc[5] - osun.W()]

In [19]:
t

cluster,L,B,D,L_err,B_err,D_err,U,V,W,U_err,V_err,W_err,UVW_dispersion,u,v,w
str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IC 348,160.4892413425805,-17.80876937457776,315.2470816561538,0.08494058638784946,0.05237350424882814,7.103391755451349,-16.701330889480783,-6.145498186382202,-7.582557065016388,1.1102263132510637,1.165861951866328,0.9606088985480086,1.874728250551058,-5.601330889480785,6.094501813617823,-0.33255706501639004
NGC 1333,158.3159702616971,-20.472365874315546,296.720765776094,0.05039372944450675,0.07191893183272671,8.862305589804084,-16.835271565656218,-10.984136986743056,-9.736973336851882,1.0129897927427556,1.603970704666745,0.7959778899041949,2.0572921870377283,-5.73527156565622,1.2558630132569686,-2.486973336851884


In [20]:
t.write("data/outputs/deconvolution_results.csv", format = "csv", overwrite=True)